In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from utils import load_MTI_embeds

In [ ]:
model_id = "/root/annotated_textual_inversion/model/sd-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = lambda images, clip_input: (images, False) # 去除原来的safety_checker

In [ ]:
# for MTI

token = "<bird>"
number = 2
weight_dir="/root/annotated_textual_inversion/embeds/bird-MTI-2-1200"
name="learned_embeds"

token = load_MTI_embeds(pipe, model_id, token, number, weight_dir, name)

In [ ]:
# for ATI/TI

token = "<CL4P>"
pipe.load_textual_inversion(model_id, token, weight_name="/root/annotated_textual_inversion/output/learned_embeds.bin")

In [ ]:
# for NTI

token = "<CL4P>"
negative_token = f'{token}_negative'
pipe.load_textual_inversion(model_id, token, weight_name="/root/annotated_textual_inversion/output/learned_embeds.bin")
pipe.load_textual_inversion(model_id, negative_token, weight_name="/root/annotated_textual_inversion/output/learned_embeds-negative.bin")

In [ ]:
# add other embeddings

pipe.load_textual_inversion(model_id, "<CL4P-0>", weight_name="/root/annotated_textual_inversion/embeds/CL4P-ATI-1200/learned_embeds-token0.bin")

In [ ]:
prompt = f"a photo of {token} bird"
# prompt = f"<CL4P-0> robot on grass, {token}"
# image = pipe(prompt, negative_prompt=negative_token, num_inference_steps=50, guidance_scale=7.5).images[0]
image = pipe(prompt, num_inference_steps=50, guidance_scale=8, num_images_per_prompt=2).images

output = Image.new('RGB', (1024, 512), (255, 255, 255))
output.paste(image[0], (0, 0))
output.paste(image[1], (512, 0))

output.save(f"output/{token}.png")